<a href="https://colab.research.google.com/github/jessica-aaao/ChordsExtractor/blob/main/ChordExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!python3 -m pip install -q -U "yt-dlp[default]"
!pip install -q -U openai-whisper
!pip install -q -U demucs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.9/171.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0

In [2]:
import requests
import json
import pandas as pd
import os
import re
import unicodedata

In [3]:
from google.colab import drive
from IPython.display import display
from bs4 import BeautifulSoup

In [4]:
!git clone  https://github.com/mikezzb/lyrics-sync.git
!git clone https://github.com/filipecalegario/ISMIR2019-Large-Vocabulary-Chord-Recognition.git

Cloning into 'lyrics-sync'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 26 (delta 1), reused 2 (delta 0), pack-reused 10 (from 1)
Receiving objects: 100% (26/26), 18.88 MiB | 12.96 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Cloning into 'ISMIR2019-Large-Vocabulary-Chord-Recognition'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 152 (delta 51), reused 135 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (152/152), 25.74 MiB | 17.47 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [5]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Common

In [6]:
class SongUrls:
    def __init__(self, name, audio, lyrics, chords):
        self.name = name
        self.audio = audio
        self.lyrics = lyrics
        self.chords = chords

    def get_name(self):
        return self.name

    def get_audio_url(self):
        return self.audio

    def get_lyrics_url(self):
        return self.lyrics

    def get_chords_url(self):
        return self.chords

In [7]:
def get_urls():
    """
    Carrega o arquivo CSV com as infformações das músicas.
    """
    print(f'Fetching urls...\n\n')

    file_path = '/content/drive/My Drive/TCC/CodeData/songs.csv'
    songs = pd.read_csv(file_path)

    print(f'Urls fetched!\n\n')

    return songs

In [8]:
def get_songs_from_csv():
    """
    Obtém as músicas, letras e cifras, além do nome da música, a partir do arquivo CSV.
    """
    songs = get_urls()

    song_urls = []

    print(f'Creating SongUrls...\n\n')

    for index, row in songs.iterrows():
        song_name = slugify(row['Song Name'])
        audio_url = row['Audio URL']
        lyrics_url = row['Lyrics URL']
        chords_url = row['Chords URL']

        song_urls.append(SongUrls(song_name, audio_url, lyrics_url, chords_url))

    print(f'SongUrls created!\n\n')

    return song_urls


In [9]:
def slugify(raw_song_name):
    """
    Converte o nome da música para um formato que pode ser usado como nome de arquivo.
    """
    song_name = raw_song_name.lower()

    song_name = unicodedata.normalize('NFKD', song_name)
    song_name = song_name.encode('ascii', 'ignore').decode('ascii')

    song_name = re.sub(r'[^a-z0-9]+', '_', song_name)

    song_name = song_name.strip('_')

    return song_name

# Sound

In [11]:
def extract_sound_recording(youtube_url, song_name):
    """
    Extrai o áudio da música a partir do vídeo do YouTube.
    """
    print(f'Extracting audio from {youtube_url}...\n\n')

    cookies_path = '/content/cookies.txt'
    output_path = f"/content/audios/{song_name}.wav"

    !yt-dlp {youtube_url} --audio-format "wav" --cookies {cookies_path} -x -o {output_path}  -q

    print(f'Audio saved as {output_path}!\n\n')

    return output_path


# Chords

In [12]:
def extract_chords_from(song_path, song_name):
    """
    Extrai a cifra, com o timestamp, a partir da música
    """
    print(f"Extracting chords from {song_path}...\n\n")

    output_folder = "/content/chords"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_path = f"{output_folder}/{song_name}.lab"

    %cd ISMIR2019-Large-Vocabulary-Chord-Recognition
    !pip install -q -r requirements.txt
    !python chord_recognition.py {song_path} {output_path}
    %cd ..

    print(f"Chords extracted to {output_path}!\n\n")

    return output_path

# Lyrics

## Lyrics Extraction from Webpage

In [22]:
def save_to_file(data, song_name):
    """
    Salva a letra em um arquivo de texto.
    """
    folder_path = "/content/lyrics"
    path = f"{folder_path}/{song_name}.txt"

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    with open(path, 'w') as file:
        file.write(data)

    print(f'Saved as {path}')

    return path

In [23]:
def extract_lyrics_from_html(html):
    """
    Extrai a letra da página HTML fornecida
    """

    print(f'Fetching lyrics...!\n\n')

    lyricsTag = html.find('div', class_='lyric-original')
    lyrics = ""

    for p in lyricsTag.find_all('p'):
        for br in p.find_all('br'):
            br.replace_with('\n')
        lyrics += p.get_text() + "\n"

    print(f'Lyrics fetched!\n\n')

    return lyrics


In [24]:
def get_lyrics_from_webpage(lyric_url, song_name):
    """
    Obtém a página web e extrai a letra
    """

    print(f'Fetching webpage {lyric_url}...\n\n')
    response = requests.get(lyric_url)

    if response.status_code == 200:
        print(f'Webpage fetched!\n\n')
        htmlContent = BeautifulSoup(response.content, 'html.parser')

        lyrics = extract_lyrics_from_html(htmlContent)
        lyrics_path = save_to_file(lyrics, song_name)

        return lyrics_path
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}\n\n")

## Lyrics Sync to Audio

In [56]:
def create_output_folder():
    """
    Cria as pastas de resultados para o lyrics-sync
    """
    output_folder = "/content/lyrics-sync/output"
    vocals_folder = output_folder + "/vocals"
    words_folder = output_folder + "/words"
    lrc_folder = output_folder + "/lrc"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if not os.path.exists(vocals_folder):
        os.makedirs(vocals_folder)

    if not os.path.exists(words_folder):
        os.makedirs(words_folder)

    if not os.path.exists(lrc_folder):
        os.makedirs(lrc_folder)

In [57]:
def get_timestamps(audio_path, lyrics_path, song_name):
    """
    Obtém os timestamps das palavras da letra
    """
    print("Installing conda...")
    !wget -c https://repo.continuum.io/archive/Anaconda3-2024.10-1-Linux-x86_64.sh
    !chmod +x Anaconda3-2024.10-1-Linux-x86_64.sh
    !bash ./Anaconda3-2024.10-1-Linux-x86_64.sh -b -f -p /usr/local
    print("Conda installed!")

    print("Installing lsync...")
    %cd /content/lyrics-sync/
    !conda env update -f environment.yml
    !source activate lsync

    print("Lsync installed!")
    from lsync import LyricsSync

    print("Extracting timestamps...")
    lsync = LyricsSync()
    words, lrc = lsync.sync(audio_path, lyrics_path)
    print("Timestamps extracted!")

    return f"/content/lyrics-sync/output/words/{song_name}.csv"
    %cd ..

In [58]:
def get_synced_lyrics(lyric_url, audio_path, song_name):
    """
    Obtém a letra sincronizada com o áudio.
    """

    print(f"Getting synced lyrics...\n\n")

    lyrics_path = get_lyrics_from_webpage(lyric_url, song_name)
    timestamps_path = get_timestamps(audio_path, lyrics_path, song_name)

    print(f"Synced lyrics ready!\n\n")

    return (lyrics_path, timestamps_path)

# Chord Sheets

## Chords Sync to Lyrics

### Chords Parsing

In [13]:
def simplify_chord(chord):
    """
    Simplifica os acordes resconhecidos pelo modelo, para acordes mais comuns
    nas cifras.
    """
    chord = chord.replace(":", "")
    chord = chord.replace("min", "m")
    chord = chord.replace("maj", "")
    chord = chord.replace("hdim7", "m7(b5)")
    chord = chord.replace("hdim", "m7(b5)")
    chord = chord.replace("sus4(b7)", "7sus4")

    return chord

In [14]:
def clean_chords(chords):
    """
    Dada uma lista de acordes (ordenados por tempo),
    remove repetições imediatas do mesmo acorde.
    """
    if not chords:
        return []
    result = [chords[0]]

    for i in range(1, len(chords)):
        current = chords[i]
        previous = result[-1]
        if current["chord"] != previous["chord"]:
            result.append(current)
        else:
            previous["end"] = current["end"]

    return result

In [21]:
def parse_chords(chords_path):
    """
    Percorre o arquivo .lab, simplifica e extrai os acordes em um dicionário.
    """
    print(f"Parsing chords from {chords_path}...")

    chords = []
    with open(chords_path, 'r', encoding='utf-8') as file:
        for line in file:
            start, end, chord = line.strip().split('\t')
            if chord == 'N':
                continue
            chord = simplify_chord(chord)
            chords.append({
                "start": float(start),
                "end": float(end),
                "chord": chord
            })

        print(f"Chords parsed!\n\n")
        print(chords)
        return clean_chords(chords)

### Lyrics Parsing

In [20]:
def parse_lyrics(timestamped_lyrics_path, lyrics_path):
    """
    Separa as letras, com timestamps, em linhas, de acordo com o arquivo da
    letra extraída pelo web scraping.
    """
    print(f"Parsing lyrics from {lyrics_path}...")

    timestamped_lyrics = pd.read_csv(timestamped_lyrics_path)
    timestamped_per_line = []

    with open(lyrics_path, 'r', encoding='utf-8') as file:
        for index, line in enumerate(file):
            line = line.strip().split()
            words_in_line = len(line)

            timestamped_words = timestamped_lyrics.iloc[:words_in_line]
            timestamped_lyrics = timestamped_lyrics.iloc[words_in_line:]
            timestamped_per_line.append([timestamped_words])

    print(f'Lyrics Parsed!\n\n')

    return timestamped_per_line

### General

In [15]:
def overlay_chords_on_transcribed(lyrics_per_line, chords):
    """
    Associa acordes às palavras da letra com base nos tempos de ACORDES.
    """
    print(f"Overlaying chords on lyrics...")

    result = []
    previous_end = 0
    for line_number, line in enumerate(lyrics_per_line):
        for df in line:
            for index, row in df.iterrows():
                start = row["start"]
                end = row["end"]
                word = row["label"]
                chord_start = None
                chord_name = None
                chord_end = None

                if previous_end is not None and start is not None and end is not None:
                    for chord_infos in chords:
                        chord_info_start = chord_infos["start"]
                        chord_info_end = chord_infos["end"]
                        chord_info_name = chord_infos["chord"]

                        if  start <= chord_info_start <= end:
                            chord_name = chord_info_name
                            chord_start = chord_info_start
                            chord_end = chord_info_end
                        elif previous_end <= chord_info_start < start:
                            word = f' {word}'
                            chord_name = chord_info_name
                            chord_start = chord_info_start
                            chord_end = chord_info_end

                result.append({
                    "word": word,
                    "start": start,
                    "end": end,
                    "chord": chord_name,
                    "chord_start": chord_start,
                    "chord_end": chord_end,
                    "line": line_number
                })
                previous_end = end

    print(f"Chords overlayed!\n\n")

    return result

In [16]:
def align_chord_over_word(word_info):
    """
    Posiciona o acorde sobre a palavra.
    """
    word = word_info["word"]
    word_start = word_info["start"]
    word_end = word_info["end"]
    chord = word_info["chord"]
    chord_start = word_info["chord_start"]
    chord_end = word_info["chord_end"]

    word_duration = word_end - word_start
    ratio = (chord_start - word_start) / word_duration

    word_len = len(word)
    word_index = int(round(ratio * (word_len - 1)))
    word_index = max(0, min(word_index, word_len - 1))

    left_padding = " " * word_index
    chord_with_padding = left_padding + chord

    if len(chord_with_padding) < word_len:
        chord_with_padding += " " * (word_len - len(chord_with_padding))

    return chord_with_padding

In [17]:
def format_transcribed_with_chords(lyrics_with_chords):
    """
    Gera o formato de saída com acordes acima das palavras.
    """
    formatted_output = []
    chord_line = []
    lyrics_line = []

    current_line = lyrics_with_chords[0]["line"]
    for word_info in lyrics_with_chords:
        if word_info["line"] != current_line:
            current_line = word_info["line"]
            formatted_output.append(" ".join(chord_line))
            formatted_output.append(" ".join(lyrics_line))
            formatted_output.append("")
            chord_line = []
            lyrics_line = []

        word = word_info["word"]
        chord = word_info["chord"]

        if chord:
            if word == " ":
                chord_line.append(chord)
            else:
                chord_line.append(align_chord_over_word(word_info))
        else:
            chord_line.append(" " * len(word))

        lyrics_line.append(word)

    # Adiciona o restante, se existir
    if chord_line or lyrics_line:
        formatted_output.append(" ".join(chord_line))
        formatted_output.append(" ".join(lyrics_line))

    return "\n".join(formatted_output)

In [18]:
def save_chords_sheet(data, song_name):
    """
    Salva a cifra com acordes em um arquivo de texto.
    """
    print(f"Saving chord sheet to {song_name}.txt...")

    folder_path = "/content/chord_sheets"
    path = f"{folder_path}/{song_name}.txt"

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    with open(path, 'w') as file:
        file.write(data)

    print(f'Saved as {path}!')

    return path

In [19]:
def format_song_with_chords(timestamped_lyrics_path, lyrics_path, chords_path, song_name):
    """
    Formata a letra com acordes no formato tradicional de cifras.
    """
    lyrics = parse_lyrics(timestamped_lyrics_path, lyrics_path)
    chords = parse_chords(chords_path)

    lyrics_with_chords = overlay_chords_on_transcribed(lyrics, chords)

    # Formatar saída com acordes e palavras
    formatted_output = format_transcribed_with_chords(lyrics_with_chords)

    save_chords_sheet(formatted_output, song_name)

    return formatted_output



---



# MAIN

In [ ]:
songs = get_songs_from_csv()
chord_sheets = []

for song in songs:
    song_name = song.get_name().replace(" ", "_")
    print(f"Generating \'{song_name}\' chord sheet...")

    song_path = extract_sound_recording(song.get_audio_url(), song_name)
    chords_path = extract_chords_from(song_path, song_name)

    create_output_folder()
    lyrics_path, lyrics_timestamped_path = get_synced_lyrics(song.get_lyrics_url(), song_path, song_name)

    formatted_chords = format_song_with_chords(lyrics_timestamped_path, lyrics_path, chords_path, song_name)
    chord_sheets.append(formatted_chords)

Fetching urls...


Urls fetched!


Creating SongUrls...


SongUrls created!


Generating 'petrolina_juazeiro' chord sheet...
